In [1]:
# !pip install flask
# !pip install Jinja2
# !pip install line-bot-sdk
# !pip install openai
# !pip install python-dotenv
# !pip install ngrok
# !pip install pyngrok

In [1]:
from flask import Flask, request, abort, jsonify
from pyngrok import ngrok
from linebot import LineBotApi, WebhookHandler
from linebot.models import (MessageEvent, TextMessage, TextSendMessage)
from linebot.exceptions import (InvalidSignatureError)
from dotenv import load_dotenv

import os
import openai
import requests
import json
import logging

In [2]:
!pip freeze > ../requirements.txt

In [3]:
os.getcwd()

'/workspaces/WhatToEatBot/code'

In [4]:
load_dotenv("../.env")

True

In [5]:
line_channel_secret = os.getenv("LINE_CHANNEL_SECRET")
line_channel_access_token = os.getenv("LINE_CHANNEL_ACCESS_TOKEN")
azure_openai_key = os.getenv("AZURE_OPENAI_KEY")
azure_openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_openai_deployment_name = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")
ngrok_auth_token = os.getenv("NGROK_AUTHTOKEN")
port_number = os.getenv("PORT_NUMBER")

# print("LINE Channel Secret: ", line_channel_secret)
# print("LINE Channel Access Token: ", line_channel_access_token)
# print("Azure OpenAI Key: ", azure_openai_key)
# print("Azure OpenAI Endpoint: ", azure_openai_endpoint)
# print("Azure OpenAI Deployment Name: ", azure_openai_deployment_name)
print("Ngrok Auth Token: ", ngrok_auth_token)
print("Port Number: ", port_number)

Ngrok Auth Token:  2iTLla5ejhzjbKHztEVRWFMmlnu_3w6VzQmGdsMqtNF2rett9
Port Number:  5000


In [6]:
ngrok.set_auth_token("ngrok_auth_token")
public_url =  ngrok.connect(5000).public_url
print(public_url)

t=2024-07-05T06:42:39+0000 lvl=warn msg="ngrok config file found at both XDG and legacy locations, using XDG location" xdg_path=/home/codespace/.config/ngrok/ngrok.yml legacy_path=/home/codespace/.ngrok2/ngrok.yml


https://4f1e-52-187-58-248.ngrok-free.app


In [7]:
# Initialize Line Bot API and Webhook Handler with the provided tokens
line_bot_api = LineBotApi(line_channel_access_token)
handler = WebhookHandler(line_channel_secret)

# Initialize Flask app
app = Flask(__name__)
ngrok.set_auth_token("ngrok_auth_token")
public_url =  ngrok.connect(5000).public_url

# Route for home page, handling both GET and POST methods
@app.route("/", methods=["GET","POST"])
def home():
    try:
        signature = request.headers["X-Line-Signature"]
        body = request.get_data(as_text=True)
        handler.handle(body, signature)
    except:
        pass
    
    return "Hello Line Chatbot"

@handler.add(MessageEvent, message=TextMessage)
def handle_text_message(event):
    text = event.message.text
    print(text)

    prompt_text = text

    response = openai.Completion.create(
        model=model_use,
        prompt=prompt_text,  
        max_tokens=1024) # max 4096

    text_out = response.choices[0].text 
    line_bot_api.reply_message(event.reply_token,
                               TextSendMessage(text=text_out))
    
print(f"To acces the Gloable link please click {public_url}")

if __name__ == "__main__":
    # Run the Flask app
    app.run(port = 5000)

/tmp/ipykernel_8885/1933662980.py:2: LineBotSdkDeprecatedIn30: Call to deprecated class LineBotApi. (Use v3 class; linebot.v3.<feature>. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  line_bot_api = LineBotApi(line_channel_access_token)
/tmp/ipykernel_8885/1933662980.py:3: LineBotSdkDeprecatedIn30: Call to deprecated class WebhookHandler. (Use 'from linebot.v3.webhook import WebhookHandler' instead. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  handler = WebhookHandler(line_channel_secret)


To acces the Gloable link please click https://632d-52-187-58-248.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [05/Jul/2024 06:42:55] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [05/Jul/2024 06:42:56] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [05/Jul/2024 06:43:14] "POST / HTTP/1.1" 200 -
